# Random Forest

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import time
from collections import defaultdict 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Nos modules
import randomforest as rf 
import params_tuning as pt

## 1. Importation des données

In [ ]:
data = pd.read_csv("data.csv")

In [ ]:
print(data.shape)
data.head()

In [ ]:
# On supprime la premiere colonne
data = data.drop(['filename'],axis=1)

# On change les noms des genres par des entiers (de 0 a 9) car notre random forest ne prend en compte que des entiers comme labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
labels = encoder.fit_transform(genre_list)
data.iloc[:, -1] = labels

In [ ]:
# On normalise le dataset
scaler = StandardScaler()
data_normalized = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## 2. Apprentissage du modèle de base
On reprend le modèle que nous avions implementé lors du précédent projet.

In [ ]:
# On separe le dataset en train set et test set (80%/20%)
data_train, data_test, label_train, label_test = train_test_split(data_normalized, labels, test_size=0.2)
print("data_train {0} | label_train {1}".format(data_train.shape, label_train.shape))
print("data_test  {0} | label_test  {1}".format(data_test.shape, label_test.shape))

In [ ]:
# Initialisation de la random forest
# par defaut n_trees = 200, n_samples = 100, n_cuts = 20, max_depth = 20
rf_classifier = rf.OurRandomForestClassifier() 

# Entrainement du modèle de base
start = time.time()
rf_classifier.fit(data_train, label_train)
end = time.time()
print("Execution time for building the forest: %f sec"%(float(end) - float(start)))

# Test de validation
our_predictions = [rf_classifier.predict(data_test[i,:]) for i in range(data_test.shape[0])] 

In [ ]:
# Comparaison avec sklearn
sklearn_rf = RandomForestClassifier(n_estimators=100, max_depth=20, max_features='sqrt')
sklearn_rf.fit(data_train, label_train)
sklearn_predictions = sklearn_rf.predict(data_test)

In [ ]:
print("Our random forest score : {} %".format(rf_classifier.score(our_predictions, label_test) * 100))  
print("Sklearn score : {} %".format(rf_classifier.score(sklearn_predictions, label_test)*100))

Il y a une très grande marge d'amélioration.

## 3. Feature selection
Un moyen d'améliorer notre modèle est de sélectionner les caractéristiques les plus discriminantes. 

* https://towardsdatascience.com/de-coding-random-forests-82d4dcbb91a1
* https://hub.packtpub.com/4-ways-implement-feature-selection-python-machine-learning/

In [ ]:
print("Shape of the dataset ", data_train.shape)
print("Size of the dataset before feature selection: %.2f MB"%(data_train.nbytes/1e6))
features_name = data.drop(['genre/label'], axis=1).columns

In [ ]:
start = time.time()
features, importances = rf_classifier.findFeatureImportance(data_train, label_train)
end = time.time()
print("Execution time to find the most important feature: %f sec"%(float(end) - float(start)))

In [ ]:
feature_importances = pd.DataFrame(zip(features_name, importances), columns = ['feature','importance']).sort_values('importance', ascending=False)

feature_importances[:10]

In [ ]:
# On selectionne les 25 meilleures
indexes = feature_importances.index[:25]

# On transforme le dataset d'entrainement (fs = feature selection)
fs_data_train = rf.transform(data_train, indexes)
fs_data_test = rf.transform(data_test, indexes)
print("Shape of the dataset ", fs_data_train.shape)
print("Size of the dataset after feature selection: %.2f MB"%(fs_data_train.nbytes/1e6))

In [ ]:
# Entrainement du modèle de base avec feature selection
start = time.time()
rf_classifier.fit(fs_data_train, label_train)
end = time.time()
print("Execution time for building the forest: %f sec"%(float(end) - float(start)))

# Test de validation
our_predictions = [rf_classifier.predict(fs_data_test[i,:]) for i in range(data_test.shape[0])] 

In [ ]:
# Comparaison avec sklearn
sklearn_rf.fit(fs_data_train, label_train)
sklearn_predictions = sklearn_rf.predict(fs_data_test)

In [ ]:
# Score
print("Our random forest score after feature selection: {} %".format(rf_classifier.score(our_predictions, label_test) * 100))
print("Sklearn score : {} %".format(rf_classifier.score(sklearn_predictions, label_test)*100))   

## 4. Réglage des hyperparamètres
Nous nous sommes fortement inspiré de la méthode décrite dans cet article [W. Koehrsen. Hyperparameter Tuning the Random Forest in Python, Janv. 2018](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)

Notre modèle a cinq paramètres, dont quatre que nous souhaitons optimiser  :
   - `n_trees` -- le nombre d'arbres de la forêt
   - `n_samples` -- le nombre de données à placer dans le noeud de chaque arbre avant qu'il ne soit partitionné
   - `n_cuts` -- le nombre de coupes à tester pour trouver la meilleure
   - `max_depth` -- la profondeur maximale de chaque arbre

Pour avoir une première idée de la meilleure combinaison d'hyperparamètres, nous allons effectuer une ... (Random Search Cross Validation). Cela consiste à tester un large choix de combinaisons qui ont été formées en tirant aléatoirement des valeurs dans une grille d'hyperparamètres.

### 2.1 Random Search Cross Validation

#### Random Hyperparameter Grid

On définit la grille pour la recherche aléatoire (Random Hyperparameter Grid) :

In [ ]:
# n_trees
n_trees = [int(x) for x in np.linspace(start = 200, stop = 600, num = 5)] 

# n_samples
n_samples = [int(x) for x in np.linspace(start = 200, stop = 500, num = 4)] # A GARDER OU PAS ?

# n_cuts 
# dans quel intervalle pourrait on tester ??
n_cuts = [int(x) for x in np.linspace(start = 10, stop = 50, num = 5)] 

# max_depth
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
# Est-ce qu'on ajoute un None comme dans l'article ? A voir selon notre random forest, est-ce qu'elle prend en compte
# un arg None pour max_depth ?

# Creation de la grille
random_grid = {'n_trees': n_trees,
                'n_samples': n_samples,
                'n_cuts': n_cuts,
                'max_depth': max_depth}

from pprint import pprint
print("Grille d'hyperparametres :\n")
pprint(random_grid)

Au lieu de tester 10 x 10 x 5 x 10 = 5000 combinaisons d'hyperparamètres, nous allons seulement en sélectionner quelques unes aléatoirement.

#### Random Search Training

On procède à la recherche randomisée sur 50 combinaisons, en utilisant une 3-fold CV

In [ ]:
# NE PAS RUN, JE N'AI PAS ENCORE TESTE CAR CA PREND ENORMEMENT DE TEMPS (> 30 MIN VOIRE PLUS)
# Definition de la recherche randomisee
rf_random = pt.RandomizedSearchCV(estimator = rf.OurRandomForestClassifier, 
                                  param_distributions = random_grid, 
                                  n_iter = 5, 
                                  cv = 3)

# Entrainement du modele
start = time.time()
rf_random.fit(data_train, label_train)
end = time.time()
print("Execution time for random search training: %f sec"%(float(end) - float(start)))

Les resultats

In [ ]:
print("La meilleure combinaison d'hyperparametres avec la recherche randomisee est :")
print(rf_random.best_params_)
print("")
print("Le score moyen du modele avec ces hyperparametres est :")
print(rf_random.best_score_)

### 2.2 Grid Search Cross Validation
Une fois qu'on connait a peu pres les meilleurs hyper-parametres
Plus d'aleatoire, on teste toutes les combinaisons